In [84]:
import csv
import json
import pandas as pd
import spacy
import collections


from bs4 import BeautifulSoup
from spacy import displacy

In [88]:
#train the model

!python -m spacy project run all

ℹ Running workflow 'all'

================================== convert ==================================
ℹ Skipping 'convert': nothing changed

=================================== train ===================================
Running command: 'c:\Users\chris\anaconda3\python.exe' -m spacy train configs/config.cfg --output training/ --paths.train corpus/train.spacy --paths.dev corpus/dev.spacy --training.eval_frequency 10 --training.patience 50 --gpu-id -1
ℹ Saving to output directory: training
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    481.86    0.00    0.00    0.00    0.00
  0      10         15.22   5694.35    0.00    0.0

[2023-04-09 13:00:06,063] [INFO] Set up nlp object from config
[2023-04-09 13:00:06,073] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-09 13:00:06,076] [INFO] Created vocabulary
[2023-04-09 13:00:06,077] [INFO] Finished initializing nlp object
[2023-04-09 13:00:14,632] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


## Evaluate model performance

In [3]:
# Load jobs

# Load job data
df = pd.read_csv("C:\\Users\\chris\\DATA417\\data\\jobs_raw.zip")

# Get just job descriptions
descriptions = df['Job Description']


In [50]:
# Function to tidy job description data
def pre_process_text(description, html=True):

    text = description

    if(html):
        soup = BeautifulSoup(description, "html.parser")
        text = soup.get_text(separator=' ')

        # Replace newline characters with spaces
        text = text.replace('\n', ' ')

    return text


# Function to get spacy doc summary stats
def get_doc_stats(doc):

    total_tokens = doc.__len__()

    total_masculine = len([ent.text for ent in doc.ents if ent.label_ == "MAS"])
    total_feminine = len([ent.text for ent in doc.ents if ent.label_ == "FEM"])
    total_exclusive = len([ent.text for ent in doc.ents if ent.label_ == "EXC"])

    result = {
        'total': total_tokens,
        'total_masculine': total_masculine,
        'total_feminine': total_feminine,
        'total_exclusive': total_exclusive,
        'pct_masculine': round(total_masculine/total_tokens, 4),
        'pct_feminine': round(total_feminine/total_tokens, 4),
        'pct_exclusive': round(total_exclusive/total_tokens, 4),
        'masc_ents': [ent.text for ent in doc.ents if ent.label_ == "MAS"],
        'fem_ents' : [ent.text for ent in doc.ents if ent.label_ == "FEM"],
        'exc_ents' : [ent.text for ent in doc.ents if ent.label_ == "EXC"]
    }

    return result

In [52]:
# Get random sample of job descriptions
description_eval_sample = pd.DataFrame(descriptions.sample(n=100, random_state=2))

# Create clean description
description_eval_sample["description_clean"] = description_eval_sample['Job Description'].apply(pre_process_text)

In [89]:
# Load NER model
nlp = spacy.load("C:\\Users\\chris\\DATA417\\ner_demo\\training\\model-best")

In [90]:
doc = nlp("Test job description")

In [93]:
doc.is_sentenced

C:\Users\chris\AppData\Local\Temp\ipykernel_23904\463249936.py:1: DeprecationWarning: [W107] The property `Doc.is_sentenced` is deprecated. Use `Doc.has_annotation("SENT_START")` instead.
  doc.is_sentenced


False

In [54]:
# Create spacy Doc object for each job description
description_eval_sample['spacy'] = description_eval_sample['description_clean'].apply(lambda row: nlp(row))

# Get summary statistics for Doc object
description_eval_sample['token_summary'] = description_eval_sample['spacy'].apply(lambda row: get_doc_stats(row))

description_eval_sample = pd.concat([description_eval_sample.drop(['token_summary'], axis=1), description_eval_sample['token_summary'].apply(pd.Series)], axis=1)

In [55]:
description_eval_sample

,Job Description,description_clean,spacy,total,total_masculine,total_feminine,total_exclusive,pct_masculine,pct_feminine,pct_exclusive,masc_ents,fem_ents,exc_ents
16220,"<div id=""jobDescriptionText"" class=""jobsearch-...",Summary An experienced creative produc...,"( , Summary, , An, experienced, creativ...",1003,37,12,1,0.0369,0.0120,0.0010,"[creative, talent, creative, creative, able, l...","[work, understand, together, present, work, Wo...",[special]
26848,"<div id=""jobDescriptionText"" class=""jobsearch-...",Overview If you are a motivated in...,"( , Overview, , If, you, are, a, mo...",625,8,5,1,0.0128,0.0080,0.0016,"[individual, driven, -OR-, driven, Proficient,...","[support, work, support, work, class]",[Disability]
26836,"<div id=""jobDescriptionText"" class=""jobsearch-...",If you have a background in sales or custo...,"( , If, you, have, a, background, in, sales...",304,4,3,0,0.0132,0.0099,0.0000,"[publication, finding, publication, Finding]","[service, work, service]",[]
22537,"<div id=""jobDescriptionText"" class=""jobsearch-...",Topflight Corporation has been providing in...,"( , Topflight, Corporation, , has, been, pro...",427,2,9,2,0.0047,0.0211,0.0047,"[Proficient, Ability]","[understand, work, service, work, understand, ...","[shrink, Special]"
16244,"<div id=""jobDescriptionText"" class=""jobsearch-...",BJ's Wholesale Club was the first retailer t...,"( , BJ, 's, Wholesale, Club, was, the, first,...",587,5,4,3,0.0085,0.0068,0.0051,"[talent, expert, test, test, data]","[service, communicate, support, support]","[sex, age, disability]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18569,"<div id=""jobDescriptionText"" class=""jobsearch-...",Eaton’s Lighting Division is currently see...,"( , Eaton, ’s, Lighting, Division, is, curr...",1223,13,12,3,0.0106,0.0098,0.0025,"[Lead, unique, Active, able, analytical, drive...","[work, work, Work, Work, Service, work, Servic...","[sex, age, disability]"
4114,"<div id=""jobDescriptionText"" class=""jobsearch-...",Accelerate Your Career Drive global techn...,"( , Accelerate, Your, Career, , Drive, glo...",1079,12,16,2,0.0111,0.0148,0.0019,"[most, data, data, best, data, Lead, data, dat...","[service, support, work, Support, class, servi...","[disability, age]"
13655,"<div id=""jobDescriptionText"" class=""jobsearch-...",Redfin is redefining real estate in the cons...,"( , Redfin, is, redefining, real, estate, in,...",528,4,5,1,0.0076,0.0095,0.0019,"[finding, data, best, data]","[service, support, Work, support, communicate]",[sick]
16305,"<div id=""jobDescriptionText"" class=""jobsearch-...","Job Location: Parsippany, N.J. . ...","( , Job, Location, :, Parsippany, ,, N.J....",474,9,7,3,0.0190,0.0148,0.0063,"[Analyst, insight, best, Lead, project, lead, ...","[course, Support, Work, support, work, Support...","[sex, age, disability]"


In [64]:
# Get top three masculine pct and feminine pct job advertisements
top_masc  = description_eval_sample.sort_values('pct_masculine', ascending=False)[0:3]
top_fem = description_eval_sample.sort_values('pct_feminine', ascending=False)[0:3]

# Create lists of terms across all named entity types
masc_ents = description_eval_sample['masc_ents'].tolist()
masc_ents = [i for j in masc_ents for i in j]

fem_ents = description_eval_sample['fem_ents'].tolist()
fem_ents = [i for j in fem_ents for i in j]

exc_ents = description_eval_sample['exc_ents'].tolist()
exc_ents = [i for j in exc_ents for i in j]


In [74]:
# Count occurrences of entities by label type
masc_ents = pd.DataFrame({'ent' : masc_ents})
masc_ents_count  = masc_ents['ent'].value_counts()

fem_ents = pd.DataFrame({'ent' : fem_ents})
fem_ents_count  = fem_ents['ent'].value_counts()

In [79]:
masc_ents_count

ability        129
Ability        121
data            85
able            66
best            49
              ... 
Most             1
logic            1
calibration      1
cohesion         1
Aptitude         1
Name: ent, Length: 110, dtype: int64

In [80]:
fem_ents_count

work                    247
service                 164
support                  81
Work                     63
Service                  44
understand               28
communicate              27
interpersonal            23
Support                  21
share                    15
feel                     13
Communicate              12
class                    11
trust                    11
inclusive                10
Understand                9
present                   8
train                     6
together                  5
disciplined               5
Present                   5
WORK                      5
course                    5
rapport                   5
connect                   4
mentor                    4
Honest                    3
warm                      3
her                       3
teach                     3
Train                     3
SERVICE                   2
kind                      2
Together                  2
Class                     2
honest              

## Masculine

In [44]:
displacy.render(top_masc['spacy'].iloc[0], style='ent')

In [45]:
displacy.render(top_masc['spacy'].iloc[1], style='ent')

In [46]:
displacy.render(top_masc['spacy'].iloc[2], style='ent')

## Feminine

In [47]:
displacy.render(top_fem['spacy'].iloc[0], style='ent')

In [48]:
displacy.render(top_fem['spacy'].iloc[1], style='ent')

In [49]:
displacy.render(top_fem['spacy'].iloc[2], style='ent')